# Functional MRI Preprocessing Steps

In [16]:
# import required packages
import os
import numpy as np
import pandas as pd

In [17]:
# function for viewing full dataframes

def print_full_df(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')


In [18]:
def append_column(df, dir, column_name, search_phrase='.nii'):
    '''
    This function appends column of filepaths to a dataframe matching by subject_id. 

    Files within folders must contain the subject id string pattern. 

    INPUTS:
        1. main_df          - the dataframe to append columns to
        2. folder_name      - folder containing files/paths to append
        3. column_name      - name of the column to append
        4. search_phrase    - text pattern to look for inside specified folder (default '.nii')

    '''
    # search directory for phrase and store as a list
    filelist = !find "{fmri_dir}/{dir}" ~+ -type f -name "*{search_phrase}*"

    # convert list to a dataframe and create a column matching subject_id
    tmp_df = pd.DataFrame({'subject_id':"", column_name:filelist})
    tmp_df['subject_id'] = tmp_df[column_name].str.extract(r'(\w{3}_\d{3}_m\d{2})')

    # append the new filepaths by matching subject_id 
    df = pd.merge(df, tmp_df, on='subject_id')

    return df

In [19]:
def append_anat_column(df, dir, column_name, search_phrase='.nii'):
    '''
    This function appends column of filepaths to a dataframe matching by subject_id. 

    Files within folders must contain the subject id string pattern. 

    INPUTS:
        1. main_df          - the dataframe to append columns to
        2. folder_name      - folder containing files/paths to append
        3. column_name      - name of the column to append
        4. search_phrase    - text pattern to look for inside specified folder (default '.nii')

    '''
    # search directory for phrase and store as a list
    filelist = !find "{anat_dir}/{dir}" ~+ -type f -name "*{search_phrase}*"

    # convert list to a dataframe and create a column matching subject_id
    tmp_df = pd.DataFrame({'subject_id':"", column_name:filelist})
    tmp_df['subject_id'] = tmp_df[column_name].str.extract(r'(\w{3}_\d{3}_m\d{2})')

    # append the new filepaths by matching subject_id 
    df = pd.merge(df, tmp_df, on='subject_id')

    return df

In [20]:
# define parent directories for func and struct data
anat_dir = '/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden'
fmri_dir = '/media/forest/wd_1/data_CLMS/functional/7T_func'

In [21]:
# create the initial dataframe with subject_id and raw data filepaths
raw_filelist = !find "{fmri_dir}/0_raw" ~+ -type f -name "*.nii*"
df_pp_fmri = pd.DataFrame({'subject_id':"", 'fp_raw':raw_filelist})
df_pp_fmri['subject_id'] = df_pp_fmri['fp_raw'].str.extract(r'(\w{3}_\d{3}_m\d{2})')

In [22]:
## Create code to extract volume #240 from each subject's raw fmri data
for subject, file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_raw']):
    print(f'fslroi {file} {fmri_dir}/3_mid_volume/{subject}_func_middle_volume 240 1 &&')

fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aab_026_m00_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aab_026_m00_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aac_027_m00_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aac_027_m00_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aac_027_m24_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aac_027_m24_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aad_028_m00_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aad_028_m00_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aad_028_m24_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aad_028_m24_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_

In [23]:
## Append to df, check shape which should be 64x3
df_pp_fmri = append_column(df_pp_fmri, '3_mid_volume', 'fp_middle_volume', '.nii')

In [24]:
## Create code to skullstrip the middle volume data--use conservative settings in BET (same as FEAT pipeline) and visually inspect results
for subject, file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_middle_volume']):
    print(f'bet2 {file} {fmri_dir}/5_mid_volume_SS/{subject}_func_middle_volume_skullstripped -f 0.3 &&')

bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aab_026_m00_func_middle_volume.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aab_026_m00_func_middle_volume_skullstripped -f 0.3 &&
bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aac_027_m00_func_middle_volume.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m00_func_middle_volume_skullstripped -f 0.3 &&
bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aac_027_m24_func_middle_volume.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m24_func_middle_volume_skullstripped -f 0.3 &&
bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aad_028_m00_func_middle_volume.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aad_028_m00_func_middle_volume_skullstripped -f 0.3 &&
bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aad_028_m24_func_middle_volume.nii.gz /med

In [25]:
## append to df, check shape which should be 64x4
df_pp_fmri = append_column(df_pp_fmri, '5_mid_volume_SS', 'fp_middle_volume_SS', '.nii')

In [26]:
df_pp_fmri

,subject_id,fp_raw,fp_middle_volume,fp_middle_volume_SS
0,aab_026_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
1,aac_027_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
2,aac_027_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
3,aad_028_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
4,aad_028_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
...,...,...,...,...
59,xyz_024_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
60,xyz_024_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
61,aal_036_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
62,bbi_059_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...


In [ ]:
# Create 

# Slice Timing Correction

- order is Siemens interleaved, which is ascending and slice order depends on number of slices:
    - if slices is even, even-first (i.e. 2, 4, 6, ... followed by 1, 3, 5...)
    - all fmri images are 56 slices in Z, so order is interleaved ascending, with even-first slice order
    - can simply use the "odd" option in slicertimer command, which will adjust based on odd/even slices
- each file takes a few minutes, but 20-30 subjects can be run in parallel
    - main bottleneck is probably disk read/write speed here

In [79]:
## generate code to apply slice timing correction and output files to 1_STC
for subject, file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_raw']):
    print(f'slicetimer -i {file} -o {fmri_dir}/1_STC/{subject}_fmri_stc -r 1.25 --odd &')

slicetimer -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aab_026_m00_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aab_026_m00_fmri_stc -r 1.25 --odd &
slicetimer -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aac_027_m00_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aac_027_m00_fmri_stc -r 1.25 --odd &
slicetimer -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aac_027_m24_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aac_027_m24_fmri_stc -r 1.25 --odd &
slicetimer -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aad_028_m00_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aad_028_m00_fmri_stc -r 1.25 --odd &
slicetimer -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aad_028_m24_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aad_028_m24_fmri_stc -r 1.25 --odd &
slicetimer -i /media/forest/wd_1/data_CLMS/functio

In [89]:
# append slicetiming corrected files to dataframe
df_pp_fmri = append_column(df_pp_fmri, '1_STC', 'fp_raw_STC', '.nii')

# Motion Correction and Outlier Identification

## Motion Correction with MCFLIRT

In [93]:
# generate code to apply motion correction to slicetiming corrected file
for subject, slicetiming_corrected_file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_raw_STC']):
    print(f'mcflirt -in {slicetiming_corrected_file} -o {fmri_dir}/4_MC/{subject}_STC_MC &')

mcflirt -in /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aab_026_m00_fmri_stc.nii.gz -o /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aab_026_m00_STC_MC &
mcflirt -in /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aac_027_m00_fmri_stc.nii.gz -o /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aac_027_m00_STC_MC &
mcflirt -in /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aac_027_m24_fmri_stc.nii.gz -o /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aac_027_m24_STC_MC &
mcflirt -in /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aad_028_m00_fmri_stc.nii.gz -o /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aad_028_m00_STC_MC &
mcflirt -in /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aad_028_m24_fmri_stc.nii.gz -o /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aad_028_m24_STC_MC &
mcflirt -in /media/forest/wd_1/data_CLMS/functional/7T_func/1_STC/aae_029_m00_fmri_stc.nii.gz -o /media/forest/wd_1/data_CLMS/functional/7T_func/

## Motion Outlier Identification using FSL_motion_outliers

In [85]:
# generate code and use the original raw image for this
for subject, file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_raw']):
    print(f'fsl_motion_outliers -i {file} -o {fmri_dir}/4_MC/{subject}_func_STC_MC &&')

fsl_motion_outliers -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aab_026_m00_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/2_MC_outliers/aab_026_m00_motion_outliers -s -p --nomoco &&
fsl_motion_outliers -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aac_027_m00_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/2_MC_outliers/aac_027_m00_motion_outliers -s -p --nomoco &&
fsl_motion_outliers -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aac_027_m24_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/2_MC_outliers/aac_027_m24_motion_outliers -s -p --nomoco &&
fsl_motion_outliers -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aad_028_m00_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS/functional/7T_func/2_MC_outliers/aad_028_m00_motion_outliers -s -p --nomoco &&
fsl_motion_outliers -i /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aad_028_m24_7T_FMRI_raw.nii -o /media/forest/wd_1/data_CLMS

In [104]:
# append motion-corrected images to df
df_pp_fmri = append_column(df_pp_fmri, '4_MC', 'func_STC_MC', 'STC_MC.nii')

In [105]:
df_pp_fmri = append_column(df_pp_fmri, '2_MC_outliers', 'motion_outliers', 'motion_outliers')

# Boundary Based Registration (fmri to anatomical transformation)

Required inputs: 
- functional image--use single-volume (volume # 240) from functional scan
    - middle volume in `func/3_mid_volume`
    - skullstripped in `func/5_mid_volume_SS`
- t1 wholehead image--use `anat/1_robustfov`
- t1 brain only--use `anat/2_skullstripped` 
- output path--use `func/7_fmri2anat_xfm`

In [27]:
# get filepaths for the T1 images and append these to the column
df_pp_fmri = append_anat_column(df_pp_fmri, '1_robustfov_fslreorient2std', 'fp_t1_wholebrain', '.nii')

In [29]:
df_pp_fmri = append_anat_column(df_pp_fmri, '2_skullstripped', 'fp_t1_skullstripped', 'BrainExtractionBrain.nii')

In [36]:
df_pp_fmri.shape

(64, 6)

In [34]:
for subject, fmri_file, t1_wholebrain, t1_skullstripped in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_middle_volume_SS'], df_pp_fmri['fp_t1_wholebrain'], df_pp_fmri['fp_t1_skullstripped']):
    print(f'epi_reg --epi={fmri_file} --t1={t1_wholebrain} --t1brain={t1_skullstripped} --out={fmri_dir}/7_fmri2anat_xfm/{subject}_fmri2anat &')

epi_reg --epi=/media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aab_026_m00_func_middle_volume_skullstripped.nii.gz --t1=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov_fslreorient2std/aab_026_m00_7T_mprage_uniden_robustfov.nii.gz --t1brain=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aab_026_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz --out=/media/forest/wd_1/data_CLMS/functional/7T_func/7_fmri2anat_xfm/aab_026_m00_fmri2anat &
epi_reg --epi=/media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m00_func_middle_volume_skullstripped.nii.gz --t1=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov_fslreorient2std/aac_027_m00_7T_mprage_uniden_robustfov.nii.gz --t1brain=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aac_027_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz --out=/media/forest/wd_1/data_CLMS

- Spot check, then copy/paste code to terminal
- Can run 15 in parallel with ~25 GB memory usage 
- spot check by overlaying resulting transformed nifti image onto the anatomical image
- generated code for fslview in below cell: 

In [38]:
# append outputs to dataframe
df_pp_fmri = append_column(df_pp_fmri, '7_fmri2anat_xfm', 'fp_fmri2anat_nifti', 'fmri2anat.nii')

In [52]:
df_pp_fmri.shape

(64, 7)

In [53]:
# for each subject, generate code to overlay the registered fmri image onto the robustfov anatomical image in fsleyes
#   - set robustfov anatomical in the background
#   - set registered fmri to red-yellow with 50% transparency

for subject, fmri_to_anat_nifti, robustfov_anat in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_fmri2anat_nifti'], df_pp_fmri['fp_t1_skullstripped']):
    print(f'FSLeyes scene for subject: {subject}:')
    print(f'''\
fsleyes --scene custom1 \
{robustfov_anat} --name {subject}_anatomical_SS \
{fmri_to_anat_nifti} --name {subject}_fmri2anat --alpha 50 --cmap red-yellow''')
    print()

FSLeyes scene for subject: aab_026_m00:
fsleyes --scene custom1 /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aab_026_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz --name aab_026_m00_anatomical_SS /media/forest/wd_1/data_CLMS/functional/7T_func/7_fmri2anat_xfm/aab_026_m00_fmri2anat.nii.gz --name aab_026_m00_fmri2anat --alpha 50 --cmap red-yellow

FSLeyes scene for subject: aac_027_m00:
fsleyes --scene custom1 /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aac_027_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz --name aac_027_m00_anatomical_SS /media/forest/wd_1/data_CLMS/functional/7T_func/7_fmri2anat_xfm/aac_027_m00_fmri2anat.nii.gz --name aac_027_m00_fmri2anat --alpha 50 --cmap red-yellow

FSLeyes scene for subject: aac_027_m24:
fsleyes --scene custom1 /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aac_027_m24_T1_mprage_uniden_skullstripped_af

- After visually inspecting each fmri-to-anatomical registration, findings are listed as follows:
    - Some subjects have excess skull in the anatomical images, as mentioned in the structural preprocessing notebook. Despite this, there was a good registration from fmri with the BBR, with the excess skull ignored. 
    - klm_011_m24 did not register properly
    - All other subjects had good registrations. 

## Manual Intervention: fmri to anatomical registration for klm_011_m24
- Only subject with manual intervention required. 
    - problem seems to be that a large chunk of brain around the cerebellum has no signal in the fmri image
    - using 3mm smoothing on the fmri prior to BBR gives a much more accurate registration--NOTE: used robustfov for the t1, and antsbrainextraction_kirby_5 for skullstrip
    - outputs from BBR:
        - subj_xfm.mat
        - subj_xfm.nii.gz
        - subj_xfm_fast_wmedge.nii.gz
        - subj_xfm_wmseg.nii.gz
        - subj_xfm_init.mat
    - the .mat file can be used with flirt to generate the fmri2atlas

In [54]:
df_pp_fmri.columns

Index(['subject_id', 'fp_raw', 'fp_middle_volume', 'fp_middle_volume_SS',
       'fp_t1_wholebrain', 'fp_t1_skullstripped', 'fp_fmri2anat_nifti'],
      dtype='object')

In [69]:
# in the subdirectory 5_mid_volume_SS, smooth the klm_011_m24 file
# run the output of the following in terminal:
for subject, file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_middle_volume_SS']):
    if subject == 'klm_011_m24':
        klm_011_m24_epi = f'{fmri_dir}/5_mid_volume_SS/{subject}_func_middle_volume_skullstripped_3mm_smoothed'
        print(f'fslmaths {file} -kernel gauss 1.73 -fmean {klm_011_m24_epi}')


fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/klm_011_m24_func_middle_volume_skullstripped.nii.gz -kernel gauss 1.73 -fmean /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/klm_011_m24_func_middle_volume_skullstripped_3mm_smoothed


In [72]:
# rerun the epi_reg with the 3mm_smoothed file for klm_011_m24
for subject, fmri_file, t1_wholebrain, t1_skullstripped in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_middle_volume_SS'], df_pp_fmri['fp_t1_wholebrain'], df_pp_fmri['fp_t1_skullstripped']):
    if subject == 'klm_011_m24':
        print(f'epi_reg --epi={klm_011_m24_epi} --t1={t1_wholebrain} --t1brain={t1_skullstripped} --out={fmri_dir}/7_fmri2anat_xfm/{subject}_fmri2anat')

epi_reg --epi=/media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/klm_011_m24_func_middle_volume_skullstripped_3mm_smoothed --t1=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov_fslreorient2std/klm_011_m24_7T_mprage_uniden_robustfov.nii.gz --t1brain=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/klm_011_m24_T1_mprage_uniden_skullstripped_kirby_5_BrainExtractionBrain.nii.gz --out=/media/forest/wd_1/data_CLMS/functional/7T_func/7_fmri2anat_xfm/klm_011_m24_fmri2anat


Visual inspection of output (with previously generated FSLeyes code):
- result = much better
- now all subjects are usable for analysis

In [95]:
df_pp_fmri.columns

Index(['subject_id', 'fp_raw', 'fp_middle_volume', 'fp_middle_volume_SS',
       'fp_t1_wholebrain', 'fp_t1_skullstripped', 'fp_fmri2anat_nifti',
       'fp_raw_STC'],
      dtype='object')

In [98]:
# Create binary mask from skullstripped functional image and apply on original raw image (NOTE: MANUALLY CHANGE KLM_011_M24)
for subject, file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_middle_volume_SS']):
    print(f'fslmaths {file} -bin {fmri_dir}/5_mid_volume_SS/{subject}_func_middle_volume_skullstripped_mask &&')

fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aab_026_m00_func_middle_volume_skullstripped.nii.gz -bin /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aab_026_m00_func_middle_volume_skullstripped_mask &&
fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m00_func_middle_volume_skullstripped.nii.gz -bin /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m00_func_middle_volume_skullstripped_mask &&
fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m24_func_middle_volume_skullstripped.nii.gz -bin /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m24_func_middle_volume_skullstripped_mask &&
fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aad_028_m00_func_middle_volume_skullstripped.nii.gz -bin /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aad_028_m00_func_middle_volume_skullstripped_mask &&
fslmaths /me

In [100]:
df_pp_fmri = append_column(df_pp_fmri, '5_mid_volume_SS', 'func_SS_mask', 'skullstripped_mask.nii')

# Apply Skullstrip Mask to STC+MC'ed 4D Image

In [109]:
for subject, func_image, mask in zip(df_pp_fmri['subject_id'], df_pp_fmri['func_STC_MC'], df_pp_fmri['func_SS_mask']):
    print(f'fslmaths {func_image} -mul {mask} {fmri_dir}/6_SS/{subject}_func_skullstripped &&')

fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aab_026_m00_STC_MC.nii.gz -mul /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aab_026_m00_func_middle_volume_skullstripped_mask.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/6_SS/aab_026_m00_func_skullstripped &&
fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aac_027_m00_STC_MC.nii.gz -mul /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m00_func_middle_volume_skullstripped_mask.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/6_SS/aac_027_m00_func_skullstripped &&
fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aac_027_m24_STC_MC.nii.gz -mul /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m24_func_middle_volume_skullstripped_mask.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/6_SS/aac_027_m24_func_skullstripped &&
fslmaths /media/forest/wd_1/data_CLMS/functional/7T_func/4_MC/aad_028_m00_STC_MC.nii.gz -mul /med

In [ ]:
# append output to dataframe

# Functional to Atlas Combined Registration Pipeline

In [ ]:
# 

## Atlas to Functional Combined Registration
- same thing as previous, but in reverse
- use this on atlases formatted with integer labels for each ROI
- may need additional steps to ensure labels are still integer values after
- Additional Requirements:
    - ITK-Snap and Convert3D (C3D) tools must be installed to convert between FSL and ANTs' .mat files if registration is to be done in a single step
    - 

In [ ]:
# convert the fmri2anat.mat file from BBR to ants format